In [13]:
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.retrievers import BM25Retriever
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts.chat import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS

In [4]:
ollama = Ollama(base_url='http://tempestade.facom.ufms.br:11435', model="phi3:medium")
similarity_model = SentenceTransformer('all-mpnet-base-v2')

In [5]:
doc_path = './resultados/EDITAL-PROAES-N-70-DE-JUNHO-DE-2024_CADASTRO-PARA-BOLSA-PERMANENCIA-DO-MEC-PARA-ESTUDANTES-INDIGENAS-E-QUILOMBOLA-MA-GRADUACAO-PROGRAMA-BPMEC-2024.pdf'

pdf_loader = PyPDFLoader(doc_path)

loaded_pdf = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=['\n', ' ', '']
)

pages = text_splitter.split_documents(loaded_pdf)

In [18]:
vectorstore = FAISS.from_documents(
    pages,
    similarity_model.encode
)

TypeError: encode() missing 1 required positional argument: 'sentences'

In [16]:
similarity_model.encode

TypeError: encode() missing 1 required positional argument: 'sentences'